In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torchvision  

import os
import os.path as osp
import torchvision.models as models

from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms
import matplotlib.pyplot as plt

In [2]:
VID_PATH = "../../data/videos/"
LABEL_PATH = "../../data/data.npy"
PROCESSED_PATH = "../../data_temp/processed"
DATA_SAVE_PATH = "../../data_temp/labeled_videos"
MODELS_PATHS = "./models"
FRAME_RATE = 2

In [3]:
"""
  Preprocess video data.
"""
import subprocess
import cv2

def map_to_multiclass(lab):
    if lab == 'LEFT':
        return 0
    if lab == 'RIGHT':
        return 1
    return 2

def get_all_files_from_dir(directory):
    file_paths = []
    print(directory)
    try:
        for root, dirs, files in os.walk(directory):
            print(files)
            file_paths += [os.path.join(root, x) for x in files]
        return sorted(file_paths)
    except Exception as e:
        print(e)
    
def get_lab(labels, time):
    for row in labels:
        if time <= float(row[2]) and time >= float(row[1]):
            return row[0]

def get_length(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

def inRange(a, b, c, mode = 'full'):
    q = round(a/1000 , 3)
    if(mode == 'full'):
        if (q>=b and q<=c):
            return True
    else:
        if(q>=b):
            return True
    return False

def process_video(video_file, label_filename):
    video_filename = video_file.split('/')[-1].split('.')[0]
    labels = pd.read_csv(label_filename, sep='\t', header=None)
    labels[1] = labels[1]-1
    labels[2] = labels[2]-1
    labels[0] = labels[0].apply(map_to_multiclass)
    labels = labels.to_numpy()

    vidcap = cv2.VideoCapture(video_file)
    # fps = vidcap.get(cv2.CAP_PROP_FPS)

    ctr = 0
    lbl = 0
    
    row = labels[lbl]
    hasFrames,image = vidcap.read()
 
    while (hasFrames and not inRange(vidcap.get(cv2.CAP_PROP_POS_MSEC), float(row[1]), float(row[2]), mode='half')):
        hasFrames,image = vidcap.read()


    video_frames = []
    video_labels = []
    while(True): 
        try:
            while(hasFrames and inRange(vidcap.get(cv2.CAP_PROP_POS_MSEC), float(row[1]),float(row[2]))):
                # savefile = {'image': image_to_save, 'label': label_to_save}
                save_file_name = video_filename + "_" + str(ctr) + ".npy"
                np.save(osp.join(PROCESSED_PATH, save_file_name), image)
                if(int(row[0]) == 1):
                    fl = 0
                elif(int(row[0]) == 0):
                    fl = 1
                else:
                    fl = 2
                
                save_file_name_flipped = video_filename+"_flip" + "_" + str(ctr) + ".npy"
                np.save(osp.join(PROCESSED_PATH, save_file_name_flipped),  cv2.flip(image, 1))
                
                
                video_labels.append(int(row[0]))
                video_frames.append(save_file_name)
                video_labels.append(fl)
                video_frames.append(save_file_name_flipped)
                
                ctr += 1
                # for _ in range(2):
                hasFrames,image = vidcap.read()
                
        except Exception as e:
            print("Error occured 1: ",e)

        if(hasFrames == False or lbl >= len(labels)-1):
            break

        lbl += 1
        row = labels[lbl]
    
    df = pd.DataFrame({'frames': video_frames, 'labels': video_labels})
    df.to_csv(osp.join(DATA_SAVE_PATH,video_filename+".csv"), index=None)

    print("After processing:")
    print("Length of labels: ",len(labels))
    print("Labels utilized: ",lbl)
    print("Frames labeled: ", len(video_frames))
    
def preprocess():
    for video_filename, label_filename in zip(get_all_files_from_dir(VID_PATH), get_all_files_from_dir(LABEL_PATH)):
        process_video(video_filename, label_filename)
        print("Finished processing ", video_filename)

In [6]:
### preprocess videos
preprocess()

../../data/Walking_with_compass/
['walking_data_3.mp4', 'walking_data_7.mp4', 'walking_data_2.mp4', 'walking_data_4.mp4', 'walking_data_1.mp4', 'walking_data_6.mp4', 'walking_data_5.mp4']
../../data/compass/
['walking_data_6_compass_label.txt', 'walking_data_2_compass_label.txt', 'walking_data_7_compass_label.txt', 'walking_data_3_compass_label.txt', 'walking_data_4_compass_label.txt', 'walking_data_1_compass_label.txt', 'walking_data_5_compass_label.txt']
['walking_data_2_compass_label-checkpoint.txt']
After processing:
Length of labels:  511
Labels utilized:  458
Frames labeled:  2782
Finished processing  ../../data/Walking_with_compass/walking_data_1.mp4
After processing:
Length of labels:  405
Labels utilized:  404
Frames labeled:  2610
Finished processing  ../../data/Walking_with_compass/walking_data_2.mp4
After processing:
Length of labels:  511
Labels utilized:  483
Frames labeled:  2962
Finished processing  ../../data/Walking_with_compass/walking_data_3.mp4
After processing:
Le

In [8]:
BATCH = 64
SEQUENCE_LENGTH = 10
HEIGHT = 128
WIDTH = 128
CHANNELS = 3

In [9]:
def save(model, index, optim = False):
    if not os.path.exists(MODELS_PATHS+'/attempt_6_frames_resnet34'):
        os.mkdir(MODELS_PATHS+'/attempt_6_frames_resnet34')
    if(optim):
        torch.save(model.state_dict(), MODELS_PATHS+'/attempt_6_frames_resnet34'+'/optimizer_params_{:08d}.pth'.format(index))
    else:
        torch.save(model.state_dict(), MODELS_PATHS+'/attempt_6_frames_resnet34'+'/model_params_{:08d}.pth'.format(index))

In [10]:
class ResNet34(nn.Module):
    """
    Container for ResNet50 s.t. it can be used for metric learning.
    The Network has been broken down to allow for higher modularity, if one wishes
    to target specific layers/blocks directly.
    """

    def __init__(self, fixconvs=False, pretrained=True):
        super(ResNet34, self).__init__()
        self.model = models.resnet34(pretrained=pretrained)
        if fixconvs:
            for param in self.model.parameters():
                param.requires_grad = False

        self.regressor = nn.Linear(self.model.fc.in_features, 3)
        self.dropout = torch.nn.Dropout(p=0.05)
        self.model = torch.nn.Sequential(*(list(self.model.children())[:-1]))
        # model.fc.weight.requires_grad = True
        # model.fc.bias.requires_grad = True

    def forward(self, x):
        x = self.model(x)
        x = torch.squeeze(x)
        x = self.dropout(x)
        x = self.regressor(x)
        return x

In [11]:
class FrameDataset(Dataset):
    def __init__(self, x, y, transforms, base_path):
        self.transforms = transforms
        self.X = x
        self.y = y
        # self.seq_len = seq_len
        self.base_path = base_path
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        seq_filename = self.X[idx]
        try:
            frame = np.load(osp.join(self.base_path,seq_filename), allow_pickle=True)
            frame = (frame - frame.min())/(frame.max() - frame.min())
            frame = self.transforms(frame)
            
        except Exception as ex:
            print("Error occured while loading frame: ", ex)
            frame = torch.zeros((CHANNELS, HEIGHT, WIDTH))
        
        return frame, self.y[idx]
        

In [12]:
def make_tt_split(data_folder):
    X = []
    y = []
    
    for filename in os.listdir(data_folder):
        if(filename[-3:]=="csv"):
            df = pd.read_csv(osp.join(data_folder,filename))
            X.append(df['frames'])
            y.append(df['labels'])
    
    
    X = pd.concat(X)
    X.reset_index(drop=True,inplace=True)
    X = X.to_numpy()

    
    y = pd.concat(y)
    y.reset_index(drop=True,inplace=True)
    y = y.to_numpy()
            
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test


In [29]:
X = []
y = []

for filename in os.listdir(data_folder):
    if(filename[-3:]=="csv"):
        df = pd.read_csv(osp.join(data_folder,filename))
        X.append(df['frames'])
        y.append(df['labels'])

X = pd.concat(X)
X.reset_index(drop=True,inplace=True)
X = X.to_numpy()
y = pd.concat(y)
y.reset_index(drop=True,inplace=True)
y = y.to_numpy()


test_dataset = FrameDataset(X, y, transforms=val_transforms, base_path = PROCESSED_PATH)
test_args = dict(shuffle=False, batch_size=BATCH, num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=BATCH, drop_last=False)
test_loader = DataLoader(test_dataset, **test_args)
print(len(test_dataset))

1311


In [13]:
cuda = torch.cuda.is_available()
print(cuda)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# train_transforms = [ttf.ToTensor(), transforms.Resize((HEIGHT, WIDTH)), transforms.ColorJitter(), transforms.RandomRotation(10), transforms.GaussianBlur(3)]
train_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((HEIGHT, WIDTH))])
val_transforms = transforms.Compose([transforms.ToTensor(), transforms.Resize((HEIGHT, WIDTH))])

X_train, X_test, y_train, y_test = make_tt_split(DATA_SAVE_PATH)
train_dataset = FrameDataset(X_train, y_train, transforms=train_transforms, base_path = PROCESSED_PATH)
val_dataset = FrameDataset(X_test, y_test, transforms=val_transforms, base_path = PROCESSED_PATH)


train_args = dict(shuffle=True, batch_size=BATCH, num_workers=1, pin_memory=True, drop_last=False) if cuda else dict(shuffle=True, batch_size=BATCH, drop_last=False)
train_loader = DataLoader(train_dataset, **train_args)

val_args = dict(shuffle=False, batch_size=BATCH, num_workers=2, pin_memory=True, drop_last=False) if cuda else dict(shuffle=False, batch_size=BATCH, drop_last=False)
val_loader = DataLoader(val_dataset, **val_args)



True


In [14]:
print(len(train_dataset))
print(len(val_dataset))

15249
3813


In [15]:
def validate(val_loader, val_dataset, model):
    # validation
    model.eval()
    val_num_correct = 0
   
    for i, (vx, vy) in enumerate(val_loader):
      
        vx = vx.float().to(device)
        vy = vy.to(device)

        with torch.no_grad():
            outputs = model(vx)
            del vx

        val_num_correct += int((torch.argmax(outputs, axis=1) == vy).sum())
        del outputs
        # break
    

    print("Validation: {:.04f}%".format(100 * val_num_correct / (len(val_dataset))))


In [26]:
lr = 0.008 #changed from 0.01
epochs = 25
lamda = 1e-3  #L2 regularization #changed from 1e-4
num_classes = 3
convlstm_hidden = 128
num_conv_lstm_layers = 2

model = ResNet34()
model.load_state_dict(torch.load('./models/attempt_6_frames_resnet34/model_params_00000022.pth'))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=lamda, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=lamda)
# optimizer.load_state_dict(torch.load('./models/attempt_6_frames_resnet34/model_params_00000022.pth'))

# for g in optimizer.param_groups:
#     g['lr'] = lr
#     g['weight_decay']= lamda
    
scaler = torch.cuda.amp.GradScaler()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# print(model)

In [30]:
# validate(val_loader, val_dataset, model)
validate(test_loader, test_dataset, model)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0c275865e0>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0c275865e0>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/anaconda3/en

Validation: 14.5690%


In [23]:
for epoch in range(11,epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0
    
    for i, (x, y) in enumerate(train_loader):
       
        model.train()
        optimizer.zero_grad()

        x = x.float().to(device)
        y = y.to(device)
        
        with torch.cuda.amp.autocast():
            outputs = model(x)
            del x
            loss = criterion(outputs.view(-1,num_classes), y.long().view(-1))

        # print(outputs.shape)
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        del outputs
        total_loss += float(loss)

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * BATCH)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        scaler.scale(loss).backward()
        scaler.step(optimizer) 
        scaler.update()

        scheduler.step()

        batch_bar.update() # Update tqdm bar
        # break
        

    batch_bar.close()

    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        100 * num_correct / (len(train_dataset)),
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    
    save(model, epoch)
    save(optimizer, epoch, optim=True)
    
    validate(val_loader, val_dataset, model)
    
batch_bar.close()

Epoch 12/25: Train Acc 89.5928%, Train Loss 0.2666, Learning Rate 0.0047
Validation: 87.9098%


Epoch 13/25: Train Acc 90.3731%, Train Loss 0.2439, Learning Rate 0.0047
Validation: 68.5812%


Epoch 14/25: Train Acc 90.6617%, Train Loss 0.2386, Learning Rate 0.0046
Validation: 83.1891%


Epoch 15/25: Train Acc 91.6060%, Train Loss 0.2216, Learning Rate 0.0045
Validation: 90.0865%


Epoch 16/25: Train Acc 92.1962%, Train Loss 0.2077, Learning Rate 0.0043
Validation: 82.2974%


...

Epoch 21/25: Train Acc 94.5046%, Train Loss 0.1467, Learning Rate 0.0031
Validation: 88.8015%


Epoch 22/25: Train Acc 95.4227%, Train Loss 0.1261, Learning Rate 0.0028
Validation: 91.3454%


Epoch 23/25: Train Acc 95.5407%, Train Loss 0.1216, Learning Rate 0.0025
Validation: 94.9908%


Epoch 24/25: Train Acc 96.1899%, Train Loss 0.1066, Learning Rate 0.0022
Validation: 93.2599%


Epoch 25/25: Train Acc 96.4719%, Train Loss 0.0990, Learning Rate 0.0019
Validation: 90.8471%
